In [1]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import math
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
train = pd.read_csv("train.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# Testua aztertzen

Testua aztertzeko listak sortuko ditugu esaldiak bilduko dituena

In [2]:
text = list(train.loc[:,"text"])
target = list(train.loc[:,"target"])
      
print(text[0:4],target[0:4])
print(len(text),len(target))
print(sum(target))

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all', 'Forest fire near La Ronge Sask. Canada', "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected", '13,000 people receive #wildfires evacuation orders in California '] [1, 1, 1, 1]
7613 7613
3271


# Testua garbitzen

Hiztegiako puntuazio markak eta beste hainak karaktere kenduko dizkiogu textuari

In [3]:
def txtclean(txt):
    a=[]
    for x in text:
        p1=re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
        p1=p1.lower()
        a.append(p1)
    return a

text=txtclean(text)
text_test=text[:600]
target_test=target[:600]
text_train=text[600:]
target_train=target[600:]

# Bi testu bektore sortzen

0 eta 1koak desberdindu behar ditugu beraz horretan arituko gara. 

In [4]:
def texttrain(txt,tar):
    train1=[]
    train0=[]
    i=0
    for text in txt:
        if tar[i]==1:
            train1.append(text)
        else:
            train0.append(text)
        i=i+1
    return(train0,train1)

train0,train1=texttrain(text_train,target_train)
print(len(text_train))
print(len(train0))
print(len(train1))


7013
3985
3028


# Hiztegiak sortzen

Daukagun testuak hiztegi bihurtuko ditugu

In [5]:
# create the transform
vectorizer = CountVectorizer(ngram_range=(1,2),max_df=0.35)
# tokenize and build vocab
vectorizer.fit(text_train)
# summarize
# encode document
vtrain0 = vectorizer.transform(train0)
vtrain1 = vectorizer.transform(train1)
vtest = vectorizer.transform(text_test)
vtrain = vectorizer.transform(text_train)

In [6]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(vtrain, target_train)
score = classifier.score(vtest, target_test)
print(score)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8033333333333333


In [7]:
from keras.models import Sequential
from keras import layers
import keras.layers as lyr
from keras.layers import Input, Embedding, LSTM, multiply, Dense
from keras.models import Model

input_tensor= Input(vtrain.shape[1:],name='esaldia')
words_embedding_layer = Embedding(vtrain.max()+1,100)
seq_embedding_layer = LSTM(256, activation='tanh')
seq_embedding=lambda tensor:seq_embedding_layer(words_embedding_layer(tensor))

merge_layer = seq_embedding(input_tensor)
dense1_layer=Dense(16,activation='sigmoid')(merge_layer)
output_layer=Dense(1,activation='sigmoid')(dense1_layer)

model=Model(input_tensor, output_layer)

model.compile(loss='binary_crossentropy',optimizer='adam')
model.summary()

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
esaldia (InputLayer)         (None, 84602)             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 84602, 100)        1400      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                4112      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 371,097
Trainable params: 371,097
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(vtrain, target_train,verbose=False, validation_data=(vtest,target_test),epochs=50,batch_size=10)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


KeyboardInterrupt: 

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(vtrain, target_train,epochs=100,verbose=False,validation_data=(vtest, target_test),batch_size=10)

In [ ]:
loss1, accuracy1 = model.evaluate(vtrain, target_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy1))
loss2, accuracy2 = model.evaluate(vtest, target_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy2))

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()


In [ ]:
plot_history(history)

In [ ]:
# summarize encoded vector
for v in [vtrain0, vtrain1, vtest] :    
    print(v.shape)
    print(type(v))
    print(v.toarray())

# Bektoreak sortzen

Testuak bektoretan bihurtu ondoren, orain bektore guztiak batuko ditugu hitz bakoitzaren maiztasuna jakiteko.

In [ ]:
vtrain1 = vtrain1.sum(axis=0)
vtrain0 = vtrain0.sum(axis=0)
for v in [vtrain1, vtrain0] :    
    print(v.shape)
    print(type(v))
    print(v)

# Maiztasun erlatiboak

Maiztasun erlatiboak kalkulatuko ditugu.

In [ ]:
ftrain1 = vtrain1/vtrain1.sum()
print(vtrain1.sum())
print(ftrain1)
print(ftrain1.sum())
print('------------------------------')
ftrain0 = vtrain0/vtrain0.sum()
print(ftrain0)
print(ftrain0.sum())
print('------------------------------')
ftest = vtest.multiply(1/(vtest.sum(axis=1))).toarray()
print(ftest,ftest.shape)

In [ ]:
for f in (ftrain1, ftrain0, ftest):
    print(f.shape,f)

# Distantziak kalkulatuko ditugu


In [ ]:
import numpy as np
#print(ftrain1.shape,ftrain1)
#print(ftrain0.shape,ftrain0)
#print(ftest.shape,ftest)

print('----')
print(np.minimum(ftrain1,ftest).shape,np.minimum(ftrain1,ftest))
atrain1 =1- np.minimum(ftrain1,ftest).sum(axis=1)

print(atrain1.shape,atrain1)
print('----')
atrain0 =1- np.minimum(ftrain0,ftest).sum(axis=1)
print(atrain0.shape,atrain0)

In [ ]:
antz=np.log(np.divide((atrain0+1e-100),(atrain1+1e-100)))
antze=antz.transpose().tolist()[0]
print(min(antze))
print(max(antze))
print(antze,len(antze))

In [ ]:
def errorea(score,target):
    z = sorted(zip(score,target))
    g2 = [ y for x,y in z ]
    s2 = [ x for x,y in z ]
    erro=[]
    eps=[]
    n = len(s2)
    for i,x in enumerate(s2):
        e1 = sum(g2[:i+1])
        e2 = n-i-1-sum(g2[i+1:])
        e = (e1+e2)/n * 100
        erro.append(e)
        eps.append(x)
    return(eps,erro)

eps,erro=errorea(antze,target_test)        
plt.plot(eps,erro)
plt.show()
epsilon=eps[erro.index(min(erro))]

print(min(erro))
print(epsilon)